In [2]:
# =======================
# 1. Imports
# =======================
import pandas as pd
import ollama
import time
import json
import re

# =======================
# 2. Load dataset
# =======================
df = pd.read_csv(r"C:\Users\amang\sample_project\Rishabh_mess.csv")  # Ensure column: "message_body"

# =======================
# 3. Define label categories
# =======================
label_categories = [
    "Phishing",
    "Smishing",
    "Promotional",
    "Loan/Financial Scam",
    "Job Scam",
    "Crypto/Investment Scam",
    "No Spam"
]

labels_str = ", ".join(label_categories)

# =======================
# 4. Define a function to classify a single SMS (with confidence)
# =======================
def classify_sms_with_confidence(text, model="deepseek-v3.1:671b-cloud", retries=3, backoff=2):
    prompt = f"""
    You are an SMS classifier. Classify the following SMS into one of these categories:
    {labels_str}.
    For each label, assign a confidence score between 0 and 1 (sum does not need to be exactly 1).

    Return ONLY a valid JSON object — no markdown, code fences, or text.
    Example output:
    {{
        "Phishing": 0.12,
        "Smishing": 0.05,
        "Promotional": 0.60,
        "Loan/Financial Scam": 0.10,
        "Job Scam": 0.05,
        "Crypto/Investment Scam": 0.02,
        "No Spam": 0.06
    }}

    SMS: "{text}"
    """

    for attempt in range(1, retries + 1):
        try:
            response = ollama.chat(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            content = response["message"]["content"].strip()

            # ---- Clean possible markdown fences ----
            content = re.sub(r"^```[a-zA-Z0-9]*", "", content).strip("` \n")

            # ---- Try to fix minor JSON issues ----
            # e.g., trailing commas, double braces, etc.
            content = re.sub(r",\s*}", "}", content)
            content = re.sub(r"}\s*}$", "}", content)

            # ---- Parse JSON safely ----
            confidences = json.loads(content)

            # Ensure all labels exist
            for label in label_categories:
                confidences.setdefault(label, 0.0)

            # ---- Determine top label ----
            predicted_label = max(confidences, key=confidences.get)
            return predicted_label, confidences

        except json.JSONDecodeError:
            print(f"⚠️ Invalid JSON on attempt {attempt}:\n{content}")
            if attempt == retries:
                return "No Spam", {label: 0.0 for label in label_categories}
            time.sleep(backoff * attempt)

        except Exception as e:
            if "502" in str(e):
                print(f"🌐 502 error on attempt {attempt} — retrying...")
                time.sleep(backoff * attempt)
                continue
            print("Error:", e)
            return "No Spam", {label: 0.0 for label in label_categories}

# =======================
# 5. Loop over dataset and classify
# =======================
predicted_labels = []
confidence_dicts = []

for idx, row in df.iterrows():
    text = str(row["message_body"])
    label, confidences = classify_sms_with_confidence(text)
    predicted_labels.append(label)
    confidence_dicts.append(confidences)
    time.sleep(0.3)  # small delay to avoid rate limits

# =======================
# 6. Merge confidence scores into DataFrame
# =======================
df["predicted_label"] = predicted_labels
conf_df = pd.DataFrame(confidence_dicts)
df = pd.concat([df, conf_df], axis=1)

# =======================
# 7. Save annotated dataset
# =======================
output_path = r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_deepseekv3_671b.csv"
df.to_csv(output_path, index=False)
print(f"✅ Annotated dataset saved successfully: {output_path}")

🌐 502 error on attempt 1 — retrying...
🌐 502 error on attempt 1 — retrying...
🌐 502 error on attempt 1 — retrying...
🌐 502 error on attempt 1 — retrying...
✅ Annotated dataset saved successfully: C:\Users\amang\sample_project\annotated_dataset_with_confidence_deepseekv3_671b.csv


In [3]:
# =======================
# 1. Imports
# =======================
import pandas as pd
import ollama
import time
import json

# =======================
# 2. Load dataset
# =======================
df = pd.read_csv(r"C:\Users\amang\sample_project\Rishabh_mess.csv")  # Ensure column: "message_body"

# =======================
# 3. Define label categories
# =======================
label_categories = [
    "Phishing",
    "Smishing",
    "Promotional",
    "Loan/Financial Scam",
    "Job Scam",
    "Crypto/Investment Scam",
    "No Spam"
]

labels_str = ", ".join(label_categories)

# =======================
# 4. Define a function to classify a single SMS (with confidence)
# =======================
def classify_sms_with_confidence(text, model="qwen3-coder:480b-cloud", retries=1):
    prompt = f"""
    You are an SMS classifier. Classify the following SMS into one of these categories:
    {labels_str}.
    For each label, assign a confidence score between 0 and 1 (sum does not need to be exactly 1).

    Return ONLY a valid JSON object — do NOT use markdown code fences, text, or explanations.
    Output must look exactly like this example:
    {{
        "Phishing": 0.12,
        "Smishing": 0.05,
        "Promotional": 0.60,
        "Loan/Financial Scam": 0.10,
        "Job Scam": 0.05,
        "Crypto/Investment Scam": 0.02,
        "No Spam": 0.06
    }}

    SMS: "{text}"
    """

    for attempt in range(retries + 1):
        try:
            response = ollama.chat(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            content = response["message"]["content"].strip()

            # ---- Clean the response (remove code fences if present) ----
            if content.startswith("```"):
                content = content.strip("`")
                content = content.replace("json", "", 1).strip()

            # ---- Parse JSON ----
            confidences = json.loads(content)

            # Ensure all labels exist in dict
            for label in label_categories:
                confidences.setdefault(label, 0.0)

            # ---- Determine top label ----
            predicted_label = max(confidences, key=confidences.get)
            return predicted_label, confidences

        except json.JSONDecodeError:
            print(f"⚠️ Invalid JSON from model (attempt {attempt+1}): {content}")
            if attempt == retries:
                # If all retries fail
                return "No Spam", {label: 0.0 for label in label_categories}
            else:
                time.sleep(1)
                continue

        except Exception as e:
            print("Error:", e)
            return "No Spam", {label: 0.0 for label in label_categories}

# =======================
# 5. Loop over dataset and classify
# =======================
predicted_labels = []
confidence_dicts = []

for idx, row in df.iterrows():
    text = str(row["message_body"])
    label, confidences = classify_sms_with_confidence(text)
    predicted_labels.append(label)
    confidence_dicts.append(confidences)
    time.sleep(0.3)  # adjust if needed for rate limits

# =======================
# 6. Merge confidence scores into DataFrame
# =======================
df["predicted_label"] = predicted_labels
conf_df = pd.DataFrame(confidence_dicts)
df = pd.concat([df, conf_df], axis=1)

# =======================
# 7. Save annotated dataset
# =======================
output_path = r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_qwen3_480b.csv"
df.to_csv(output_path, index=False)
print(f"✅ Annotated dataset saved successfully: {output_path}")

✅ Annotated dataset saved successfully: C:\Users\amang\sample_project\annotated_dataset_with_confidence_qwen3_480b.csv


In [4]:
# =======================
# 1. Imports
# =======================
import pandas as pd
import ollama
import time
import json

# =======================
# 2. Load dataset
# =======================
df = pd.read_csv(r"C:\Users\amang\sample_project\Rishabh_mess.csv")  # Ensure column: "message_body"

# =======================
# 3. Define label categories
# =======================
label_categories = [
    "Phishing",
    "Smishing",
    "Promotional",
    "Loan/Financial Scam",
    "Job Scam",
    "Crypto/Investment Scam",
    "No Spam"
]

labels_str = ", ".join(label_categories)

# =======================
# 4. Define a function to classify a single SMS (with confidence)
# =======================
def classify_sms_with_confidence(text, model="gpt-oss:20b-cloud", retries=1):
    prompt = f"""
    You are an SMS classifier. Classify the following SMS into one of these categories:
    {labels_str}.
    For each label, assign a confidence score between 0 and 1 (sum does not need to be exactly 1).

    Return ONLY a valid JSON object — do NOT use markdown code fences, text, or explanations.
    Output must look exactly like this example:
    {{
        "Phishing": 0.12,
        "Smishing": 0.05,
        "Promotional": 0.60,
        "Loan/Financial Scam": 0.10,
        "Job Scam": 0.05,
        "Crypto/Investment Scam": 0.02,
        "No Spam": 0.06
    }}

    SMS: "{text}"
    """

    for attempt in range(retries + 1):
        try:
            response = ollama.chat(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            content = response["message"]["content"].strip()

            # ---- Clean the response (remove code fences if present) ----
            if content.startswith("```"):
                content = content.strip("`")
                content = content.replace("json", "", 1).strip()

            # ---- Parse JSON ----
            confidences = json.loads(content)

            # Ensure all labels exist in dict
            for label in label_categories:
                confidences.setdefault(label, 0.0)

            # ---- Determine top label ----
            predicted_label = max(confidences, key=confidences.get)
            return predicted_label, confidences

        except json.JSONDecodeError:
            print(f"⚠️ Invalid JSON from model (attempt {attempt+1}): {content}")
            if attempt == retries:
                # If all retries fail
                return "No Spam", {label: 0.0 for label in label_categories}
            else:
                time.sleep(1)
                continue

        except Exception as e:
            print("Error:", e)
            return "No Spam", {label: 0.0 for label in label_categories}

# =======================
# 5. Loop over dataset and classify
# =======================
predicted_labels = []
confidence_dicts = []

for idx, row in df.iterrows():
    text = str(row["message_body"])
    label, confidences = classify_sms_with_confidence(text)
    predicted_labels.append(label)
    confidence_dicts.append(confidences)
    time.sleep(0.3)  # adjust if needed for rate limits

# =======================
# 6. Merge confidence scores into DataFrame
# =======================
df["predicted_label"] = predicted_labels
conf_df = pd.DataFrame(confidence_dicts)
df = pd.concat([df, conf_df], axis=1)

# =======================
# 7. Save annotated dataset
# =======================
output_path = r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_gptoss_20b.csv"
df.to_csv(output_path, index=False)
print(f"✅ Annotated dataset saved successfully: {output_path}")

✅ Annotated dataset saved successfully: C:\Users\amang\sample_project\annotated_dataset_with_confidence_gptoss_20b.csv


In [5]:
# =======================
# 1. Imports
# =======================
import pandas as pd
import ollama
import time
import json

# =======================
# 2. Load dataset
# =======================
df = pd.read_csv(r"C:\Users\amang\sample_project\Rishabh_mess.csv")  # Ensure column: "message_body"

# =======================
# 3. Define label categories
# =======================
label_categories = [
    "Phishing",
    "Smishing",
    "Promotional",
    "Loan/Financial Scam",
    "Job Scam",
    "Crypto/Investment Scam",
    "No Spam"
]

labels_str = ", ".join(label_categories)

# =======================
# 4. Define a function to classify a single SMS (with confidence)
# =======================
def classify_sms_with_confidence(text, model="gpt-oss:120b-cloud", retries=1):
    prompt = f"""
    You are an SMS classifier. Classify the following SMS into one of these categories:
    {labels_str}.
    For each label, assign a confidence score between 0 and 1 (sum does not need to be exactly 1).

    Return ONLY a valid JSON object — do NOT use markdown code fences, text, or explanations.
    Output must look exactly like this example:
    {{
        "Phishing": 0.12,
        "Smishing": 0.05,
        "Promotional": 0.60,
        "Loan/Financial Scam": 0.10,
        "Job Scam": 0.05,
        "Crypto/Investment Scam": 0.02,
        "No Spam": 0.06
    }}

    SMS: "{text}"
    """

    for attempt in range(retries + 1):
        try:
            response = ollama.chat(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            content = response["message"]["content"].strip()

            # ---- Clean the response (remove code fences if present) ----
            if content.startswith("```"):
                content = content.strip("`")
                content = content.replace("json", "", 1).strip()

            # ---- Parse JSON ----
            confidences = json.loads(content)

            # Ensure all labels exist in dict
            for label in label_categories:
                confidences.setdefault(label, 0.0)

            # ---- Determine top label ----
            predicted_label = max(confidences, key=confidences.get)
            return predicted_label, confidences

        except json.JSONDecodeError:
            print(f"⚠️ Invalid JSON from model (attempt {attempt+1}): {content}")
            if attempt == retries:
                # If all retries fail
                return "No Spam", {label: 0.0 for label in label_categories}
            else:
                time.sleep(1)
                continue

        except Exception as e:
            print("Error:", e)
            return "No Spam", {label: 0.0 for label in label_categories}

# =======================
# 5. Loop over dataset and classify
# =======================
predicted_labels = []
confidence_dicts = []

for idx, row in df.iterrows():
    text = str(row["message_body"])
    label, confidences = classify_sms_with_confidence(text)
    predicted_labels.append(label)
    confidence_dicts.append(confidences)
    time.sleep(0.3)  # adjust if needed for rate limits

# =======================
# 6. Merge confidence scores into DataFrame
# =======================
df["predicted_label"] = predicted_labels
conf_df = pd.DataFrame(confidence_dicts)
df = pd.concat([df, conf_df], axis=1)

# =======================
# 7. Save annotated dataset
# =======================
output_path = r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_gptoss_120b.csv"
df.to_csv(output_path, index=False)
print(f"✅ Annotated dataset saved successfully: {output_path}")

✅ Annotated dataset saved successfully: C:\Users\amang\sample_project\annotated_dataset_with_confidence_gptoss_120b.csv


In [6]:
# =======================
# 1. Imports
# =======================
import pandas as pd
from collections import Counter

# =======================
# 2. Define file paths
# =======================
file_paths = [
    r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_deepseekv3_671b.csv",
    r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_qwen3_480b.csv",
    r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_gptoss_20b.csv",
    r"C:\Users\amang\sample_project\annotated_dataset_with_confidence_gptoss_120b.csv"
]

# =======================
# 3. Load all datasets
# =======================
datasets = [pd.read_csv(path) for path in file_paths]

# Ensure all datasets have same order of messages
for i in range(1, len(datasets)):
    if not datasets[i]["message_body"].equals(datasets[0]["message_body"]):
        print("⚠️ Warning: Message order mismatch! You may need to merge using a common key.")
        break

# =======================
# 4. Extract label categories
# =======================
label_columns = [col for col in datasets[0].columns if col not in ["message_body", "predicted_label"]]

# =======================
# 5. Combine predictions
# =======================
final_labels = []
final_confidences = []

for i in range(len(datasets[0])):
    message = datasets[0].loc[i, "message_body"]

    # Collect predictions from each dataset
    preds = [df.loc[i, "predicted_label"] for df in datasets]

    # Majority voting
    label_counts = Counter(preds)
    most_common_label, most_common_count = label_counts.most_common(1)[0]

    # In case of tie → use average confidence
    top_labels = [label for label, count in label_counts.items() if count == most_common_count]

    if len(top_labels) > 1:
        avg_confidences = {label: 0.0 for label in label_columns}
        for df in datasets:
            for label in label_columns:
                avg_confidences[label] += df.loc[i, label]
        # Average
        for label in avg_confidences:
            avg_confidences[label] /= len(datasets)

        # Pick the label with highest avg confidence among tied ones
        most_common_label = max(top_labels, key=lambda l: avg_confidences[l])

    # Compute final average confidence of the selected label across models
    conf_values = [df.loc[i, most_common_label] for df in datasets if most_common_label in df.columns]
    final_confidence = sum(conf_values) / len(conf_values)

    final_labels.append(most_common_label)
    final_confidences.append(final_confidence)

# =======================
# 6. Build final dataset
# =======================
final_df = pd.DataFrame({
    "message_body": datasets[0]["message_body"],
    "final_predicted_label": final_labels,
    "final_confidence": final_confidences
})

# =======================
# 7. Save final dataset
# =======================
output_path = r"C:\Users\amang\sample_project\Combination_of_4.csv"
final_df.to_csv(output_path, index=False)
print(f"✅ Final prediction dataset saved as: {output_path}")

✅ Final prediction dataset saved as: C:\Users\amang\sample_project\Combination_of_4.csv
